In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
import pickle

In [2]:
SR_pic_file_path = '/kaggle/input/imd-gridded-vanilla/data/data.pkl'
with open(SR_pic_file_path, 'rb') as file:
    SR_pic_data = pickle.load(file)

In [3]:
for key in SR_pic_data:
    rainfall_array_final = [] 
    for j in range(SR_pic_data[key].shape[0]):
        k = SR_pic_data[key][j][1:,:]
        k = k[:,3:-4]
        rainfall_array_final.append(k)
    rainfall_array_final = np.array(rainfall_array_final)
    SR_pic_data[key] = rainfall_array_final

In [4]:
print(SR_pic_data[1901][0].shape)

(128, 128)


In [5]:
def calculate_daily_max(SR_data):
    daily_max = {year: [0] * SR_data[year].shape[0] for year in range(1901, 2024)}

    for year in range(1901, 2024):
        for i in range(SR_data[year].shape[0]):
            daily_max[year][i] = np.nanmax(SR_data[year][i])

    return daily_max

def normalize_with_daily_max(SR_data, daily_max):
    for year in range(1901, 2024):
        for i in range(SR_data[year].shape[0]):
            max_value = daily_max[year][i]
            if max_value != 0:
                SR_data[year][i] /= max_value
                SR_data[year][i]+=1
            SR_data[year][i][np.isnan(SR_data[year][i])] = 0
    return SR_data

def Dataprocessing(SR_data):
    daily_max = calculate_daily_max(SR_data)
    SR_data = normalize_with_daily_max(SR_data, daily_max)
    return daily_max

In [6]:
daily_max = Dataprocessing(SR_pic_data)

In [7]:
data = []
for year in SR_pic_data:
    for i in range(SR_pic_data[year].shape[0]):
        data.append(SR_pic_data[year][i])
data=np.array(data)

In [8]:
latent_dim = 256

# Encoder
input_layer = layers.Input(shape=(128, 128, 1))
x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(input_layer)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Conv2D(256, (3, 3), padding='same', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Conv2D(512, (3, 3), padding='same', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2))(x)


x = layers.Flatten()(x)

# Latent space representation
z = layers.Dense(latent_dim, name='latent_space')(x)

# Decoder
x = layers.Dense(8 * 8 * 512, activation='relu')(z)
x = layers.Reshape((8, 8, 512))(x)

x = layers.Conv2DTranspose(512, (3, 3), padding='same', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.UpSampling2D((2, 2))(x)

x = layers.Conv2DTranspose(256, (3, 3), padding='same', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.UpSampling2D((2, 2))(x)

x = layers.Conv2DTranspose(128, (3, 3), padding='same', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.UpSampling2D((2, 2))(x)

x = layers.Conv2DTranspose(64, (3, 3), padding='same', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2DTranspose(1, (3, 3), padding='same', activation='relu')(x)
autoencoder = models.Model(input_layer, decoded, name="autoencoder")
# def ssim_loss(y_true, y_pred):
#     return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, max_val=1.0))
def combined_loss(y_true, y_pred):
    ssim_loss = 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, max_val=1.0))
    mse_loss = tf.reduce_mean(tf.square(y_true - y_pred))
    return 0.9 * ssim_loss + 0.1* mse_loss
autoencoder.compile(optimizer='adam', loss=combined_loss,metrics = ['mse'])
autoencoder.summary()

Model: "autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 128, 128, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 128, 128, 64)   │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64, 64, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 32, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32, 32, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 16, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 16, 16, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 8, 8, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ latent_space (Dense)            │ (None, 256)            │     8,388,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32768)          │     8,421,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 8, 8, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose                │ (None, 8, 8, 512)      │     2,359,808 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 8, 8, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d (UpSampling2D)    │ (None, 16, 16, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_1              │ (None, 16, 16, 256)    │     1,179,904 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 22,276,865 (84.98 MB)

 Trainable params: 22,273,025 (84.96 MB)

 Non-trainable params: 3,840 (15.00 KB)

In [ ]:
autoencoder.fit(data,data,epochs=20,batch_size=32)

Epoch 1/20


I0000 00:00:1724495341.314585     105 service.cc:145] XLA service 0x7d9f88003e50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1724495341.315550     105 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
W0000 00:00:1724495342.154187     105 assert_op.cc:38] Ignoring Assert operator compile_loss/combined_loss/SSIM/Assert/Assert
W0000 00:00:1724495342.155487     105 assert_op.cc:38] Ignoring Assert operator compile_loss/combined_loss/SSIM/Assert_1/Assert
W0000 00:00:1724495342.156556     105 assert_op.cc:38] Ignoring Assert operator compile_loss/combined_loss/SSIM/Assert_2/Assert
W0000 00:00:1724495342.157200     105 assert_op.cc:38] Ignoring Assert operator compile_loss/combined_loss/SSIM/Assert_3/Assert


   2/1404 ━━━━━━━━━━━━━━━━━━━━ 1:13 53ms/step - loss: 0.8847 - mse: 1.2171  

I0000 00:00:1724495350.597091     105 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1403/1404 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.1247 - mse: 0.0426

W0000 00:00:1724495417.396585     102 assert_op.cc:38] Ignoring Assert operator compile_loss/combined_loss/SSIM/Assert/Assert
W0000 00:00:1724495417.396840     102 assert_op.cc:38] Ignoring Assert operator compile_loss/combined_loss/SSIM/Assert_1/Assert
W0000 00:00:1724495417.397162     102 assert_op.cc:38] Ignoring Assert operator compile_loss/combined_loss/SSIM/Assert_2/Assert
W0000 00:00:1724495417.397402     102 assert_op.cc:38] Ignoring Assert operator compile_loss/combined_loss/SSIM/Assert_3/Assert


1404/1404 ━━━━━━━━━━━━━━━━━━━━ 90s 52ms/step - loss: 0.1246 - mse: 0.0425
Epoch 2/20
1404/1404 ━━━━━━━━━━━━━━━━━━━━ 66s 47ms/step - loss: 0.0565 - mse: 0.0012
Epoch 3/20
1404/1404 ━━━━━━━━━━━━━━━━━━━━ 66s 47ms/step - loss: 0.0514 - mse: 0.0011
Epoch 4/20
1404/1404 ━━━━━━━━━━━━━━━━━━━━ 66s 47ms/step - loss: 0.0462 - mse: 8.2170e-04
Epoch 5/20
1404/1404 ━━━━━━━━━━━━━━━━━━━━ 66s 47ms/step - loss: 0.0438 - mse: 7.6755e-04
Epoch 6/20
1404/1404 ━━━━━━━━━━━━━━━━━━━━ 66s 47ms/step - loss: 0.0418 - mse: 6.9847e-04
Epoch 7/20
1404/1404 ━━━━━━━━━━━━━━━━━━━━ 66s 47ms/step - loss: 0.0397 - mse: 6.6540e-04
Epoch 8/20
 484/1404 ━━━━━━━━━━━━━━━━━━━━ 43s 47ms/step - loss: 0.0377 - mse: 6.2022e-04

In [ ]:
predict = autoencoder(SR_pic_data[1950][0].reshape((1,128,128,1)))
predict = np.array(predict)
lon = np.load('/kaggle/input/coordinates/0.25lon.npy')
lat = np.load('/kaggle/input/coordinates/0.25lat.npy')
lon = lon[4:-4]
X_SR, Y_SR = np.meshgrid(lon, lat)
fig, axes = plt.subplots(1, 2, figsize=(20, 6))
im1 = axes[0].contourf(X_SR, Y_SR,SR_pic_data[1950][0], cmap='Blues')
axes[0].set_title('Original')
axes[0].set_xlabel('Longitude')
axes[0].set_ylabel('Latitude')
fig.colorbar(im1, ax=axes[0], label='Rainfall (mm/day)')
im2 = axes[1].contourf(X_SR, Y_SR, predict.reshape(128,128), cmap='Blues')
axes[1].set_title('Reconstructed')
axes[1].set_xlabel('Longitude')
axes[1].set_ylabel('Latitude')
fig.colorbar(im2, ax=axes[1], label='Rainfall (mm/day)')
plt.show()

In [ ]:
autoencoder.save('autoencoder_fe.keras')